# Libs

In [2]:
# Điều hướng ra folder chủ
import os
while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
    os.chdir('..')
    print(os.getcwd())

c:\Users\Admin\Desktop\DANG NHI\repos\master-llm-rag-vnlaw


In [1]:
import ollama
import asyncio
from ollama import AsyncClient

# Create ollama client

In [3]:
ollama_api_url = "https://9e7d-34-23-184-122.ngrok-free.app"

In [4]:
ollama_client = ollama.Client(
    host=ollama_api_url,
    headers = {'Header': 'application/json'}
)

In [47]:
system_prompt = "Tôi có một sơ bộ dữ liệu gồm các lĩnh vực pháp luật Việt Nam bao gồm Bảo Hiểm, Dân sự, Doanh Nghiệp và Giáo dục. " \
"Trong đó bao bồm các văn bản hành chính của Chính phủ.\n"

agent_judgment1 = 'Bạn là một agent dùng để đánh giá câu trả lời. ' \
'Với câu hỏi "{question}", Bạn có thể trả lời câu hỏi bằng cơ sở dữ liệu của tôi được hay không (trả lời 1 nếu có thể, trả lời 0 nếu không thế).' \
'Trả lời:' \

agent_search_plan = 'Bạn là một agent để dự đoán thông tin cần truy vấn.\n' \
'Theo bạn để trả lời câu hỏi "{question}", thì cần những thông tin gì và đến từ Lĩnh vực nào?\n' \
'Trả lời tối đa 3 dòng và theo mẫu <linh_vuc>: <thong_tin_can>, linh_vuc chỉ duy nhất một, 3 câu trả lời cho thể trung Lĩnh vực nhưng khác nhau thông tin cần.\n' \
'Trả lời: '

agent_judgment2 = 'Bạn là một agent reranking' \



question = "Bảng giá dịch vụ khám chữa bệnh có bảo hiểm"

# PROMPT = agent_search_plan.format(question=question)

PROMPT = system_prompt + agent_search_plan.format(question=question)
print(PROMPT)

response = ollama_client.chat(model='gemma3:12b', messages=[
  {
    'role': 'user',
    'content': PROMPT,
    # 'stream': True
  },
], stream=False)

Tôi có một sơ bộ dữ liệu gồm các lĩnh vực pháp luật Việt Nam bao gồm Bảo Hiểm, Dân sự, Doanh Nghiệp và Giáo dục. Trong đó bao bồm các văn bản hành chính của Chính phủ.
Bạn là một agent để dự đoán thông tin cần truy vấn.
Theo bạn để trả lời câu hỏi "Bảng giá dịch vụ khám chữa bệnh có bảo hiểm", thì cần những thông tin gì và đến từ Lĩnh vực nào?
Trả lời tối đa 3 dòng và theo mẫu <linh_vuc>: <thong_tin_can>, linh_vuc chỉ duy nhất một, 3 câu trả lời cho thể trung Lĩnh vực nhưng khác nhau thông tin cần.
Trả lời: 


ResponseError: ngrok gateway error
The server returned an invalid or incomplete HTTP response.

ERR_NGROK_3004
 (status code: 503)

In [ ]:
print(response.message.content)

*   **Bảo Hiểm:** Thông tin về biểu mẫu, điều khoản, phạm vi chi trả của các gói bảo hiểm y tế, bảng giá dịch vụ được quỹ bảo hiểm thanh toán.
*   **Dân sự:** Các quy định về trách nhiệm bồi thường thiệt hại liên quan đến dịch vụ y tế, các thỏa thuận về chi phí khám chữa bệnh có bảo hiểm.
*   **Doanh Nghiệp:** Thông tin về các hợp đồng dịch vụ khám bệnh giữa bệnh viện/phòng khám và công ty bảo hiểm, bảng giá dịch vụ đã ký kết.


Tôi có một sơ bộ dữ liệu gồm các lĩnh vực pháp luật Việt Nam bao gồm Bảo Hiểm, Dân sự, Doanh Nghiệp và Giáo dục. Trong đó bao bồm các văn bản hành chính của Chính phủ.
Theo bạn để trả lời câu hỏi "Bảng giá dịch vụ khám chữa bệnh có bảo hiểm", thì cần những thông tin gì và đến từ Lĩnh vực nào?
Trả lời tối đa 3 dòng và theo mẫu <linh_vuc>: <thong_tin_can>, linh_vuc chỉ duy nhất một, 3 câu trả lời cho thể trung Lĩnh vực nhưng khác nhau thông tin cần.
Trả lời: 
